- built-in point cloud object? use bpy.ops.object.pointcloud_add
- http://georglempe.de/wordpress/index.php/2020/08/rendering-point-clouds-in-blender/
- 

# PostGIS Geometry to Blender

In [1]:
import os
from dotenv import load_dotenv
from shapely import wkt
import pdal
import psycopg2
import json
import pandas as pd
from dotenv import dotenv_values

ModuleNotFoundError: No module named 'pdal'

In [2]:
# load environment variables
load_dotenv(".env.testing")
conn_vars = ['PG_USER', 'PG_PASS', 'PG_HOST', 'PG_PORT', 'PG_DB']
user, password, host, port, dbname = [os.getenv(var) for var in conn_vars]
conn_string = f'dbname={dbname} user={user} password={password} host={host} port={port}'

In [89]:
# make database connection
conn = psycopg2.connect(conn_string, cursor_factory=psycopg2.extras.NamedTupleCursor)
cursor = conn.cursor()

In [13]:
# WKT string representing area of interest
aoi = "Polygon ((985474.64370176 186719.01067065, 985563.13880442 186665.99333437, 985490.9873919 186556.76964911, 985420.82911231 186599.42269409, 985427.60576432 186622.54303622, 985474.64370176 186719.01067065))"

In [14]:
aoi_polygon = wkt.loads(aoi)
aoi_center = aoi_polygon.centroid.coords.xy

## Tax Lots

In [74]:
cursor.execute("SELECT * FROM swimmingpool LIMIT 5;")

In [75]:
results = cursor.fetchall()

In [78]:
results[0].ogc_fid

1

## Point Cloud

In [20]:
# get lidar points using PDAL
pipeline_def = {
    "pipeline": [
        {
            "type":"readers.pgpointcloud",
            "connection": f'host={host} dbname={dbname} user={user} password={password} port={port}',
            "table": "pointcloud",
            "column": "pa",
            "spatialreference": "EPSG:2263",
            "where": f"PC_Intersects(pa, ST_GeomFromText('{aoi}',2263))"
        },
        {
            "type":"filters.crop",
            "polygon": f"{aoi}",
            "distance": 500
        },
        {   "type":"filters.hag_nn"},
        {   "type":"filters.eigenvalues",
            "knn":16},
        {   "type":"filters.normal",
            "knn":16}
    ]
}
pipeline = pdal.Pipeline(json.dumps(pipeline_def))
pipeline.validate()
%time pipeline.execute()

CPU times: user 391 ms, sys: 45.6 ms, total: 437 ms
Wall time: 2.01 s


16115

In [23]:
def pipeline_to_dataframe(pipeline):
    arr = pipeline.arrays[0]
    description = arr.dtype.descr
    cols = [col for col, __ in description]
    df = pd.DataFrame({col: arr[col] for col in cols})
    return df

In [24]:
df = pipeline_to_dataframe(pipeline)

In [28]:
# "zero" all coordinates for visualization
df['X_0'] = df['X'] - aoi_center[0]
df['Y_0'] = df['Y'] - aoi_center[1]
df['Z_0'] = df['Z'] - df['Z'].min() # uncomment to bring Z values to origin

In [29]:
df

,X,Y,Z,Intensity,ReturnNumber,NumberOfReturns,ScanDirectionFlag,EdgeOfFlightLine,Classification,ScanAngleRank,...,Eigenvalue0,Eigenvalue1,Eigenvalue2,NormalX,NormalY,NormalZ,Curvature,X_0,Y_0,Z_0
0,985422.10,186600.75,49.41,11037,1,1,0,0,2,8.922,...,0.002487,0.940527,2.153609,-0.014477,-0.001749,0.999894,0.000803,-65.685097,-36.868519,7.97
1,985422.16,186600.25,49.51,14625,1,1,0,0,1,2.952,...,0.002759,0.791253,2.032378,-0.009946,-0.012073,0.999878,0.000976,-65.625097,-37.368519,8.07
2,985422.24,186598.78,49.47,12969,1,1,1,0,1,5.202,...,0.002930,0.890157,1.656769,0.005803,-0.004145,0.999975,0.001149,-65.545097,-38.838519,8.03
3,985422.65,186599.53,49.38,12417,1,1,0,0,2,6.690,...,0.002736,0.795439,1.835969,-0.009411,-0.013664,0.999862,0.001039,-65.135097,-38.088519,7.94
4,985422.75,186602.64,49.47,11313,1,1,0,0,2,8.904,...,0.002719,1.157059,2.516748,0.012047,-0.025433,0.999604,0.000739,-65.035097,-34.978519,8.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16110,985495.18,186705.93,52.49,20419,1,1,1,0,1,-14.652,...,0.008433,0.153189,0.594060,0.073000,0.024508,0.997031,0.011160,7.394903,68.311481,11.05
16111,985495.34,186705.96,52.36,19592,1,1,0,0,1,-14.658,...,0.008433,0.153189,0.594060,0.073000,0.024508,0.997031,0.011160,7.554903,68.341481,10.92
16112,985495.54,186706.18,52.39,17618,1,1,1,0,1,15.054,...,0.008433,0.153189,0.594060,0.073000,0.024508,0.997031,0.011160,7.754903,68.561481,10.95
16113,985495.41,186706.28,52.39,19754,1,1,0,0,1,15.054,...,0.008433,0.153189,0.594060,0.073000,0.024508,0.997031,0.011160,7.624903,68.661481,10.95


## Building Blender Geometries

This part likely needs to happen on the Blender side. See https://b3d.interplanety.org/en/how-to-create-mesh-through-the-blender-python-api/

maybe use [compas](https://github.com/compas-dev/compas)?

In [ ]:
# import bpy

In [6]:
vertices = [(0, 0, 0)]
edges = []
faces = []

In [7]:
# build up mesh from verts edges and faces
# mesh = bpy.data.meshes.new('mesh1')
# mesh.from_pydata(vertices, edges, faces)
# mesh.update()

In [8]:
# wrap the mesh in a new object
# obj = bpy.data.objects.new('object1', mesh)

In [9]:
# add to the scene graph
# bpy.context.scene.collection.objects.link(obj)